In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('jmantobedropped.xls')
df

,_id_x,userId_x,projectType_x,projectId_x,projectName_x,task,comment,totalHours,timesheetId,feedbackId,...,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12
0,66166cc7d31405dcf870afd3,1,Full Stack Web Development,FULL01,Railway Ticket Booking System,"Write unit tests for backend APIs, ensure cod...",good,40,TS01,F01,...,5,8,7,6,7,2,8,8,8,8
1,66166cc7d31405dcf870afd7,1,Data Engineering,DATA01,DataNexus: Bridging Data Gaps for Holistic Bus...,Define Data Lake Requirements,awesome,47,TS02,F02,...,3,8,7,9,8,6,6,6,4,1
2,6618e3f4e7506de234ed1b8c,83,Full Stack Web Development,FULL01,Railway Ticket Booking System,"Design the database schema, define tables, and...",patch management: applied security patches.,49,TS03,F03,...,10,9,2,7,10,4,1,1,7,9
3,6618e3f4e7506de234ed1b8d,83,Full Stack Web Development,FULL01,Railway Ticket Booking System,"Design the database schema, define tables, and...",accessibility audit: fixed wcag compliance iss...,42,TS04,F04,...,8,7,1,4,8,3,1,3,7,5
4,6618e3f4e7506de234ed1b8e,83,Full Stack Web Development,FULL01,Railway Ticket Booking System,"Design the database schema, define tables, and...",sprint retrospective: identified areas for imp...,43,TS05,F05,...,6,6,7,5,8,9,10,8,6,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4274,6618e3f4e7506de234ed2c3c,442,Data Science,DASC17,Customer Sentiment Analysis for Retail And Sales,Ensuring compliance with consumer data protect...,patch management: applied security patches.,32,TS4275,F4275,...,4,2,8,10,10,10,9,9,4,10
4275,6618e3f4e7506de234ed2c3d,442,Data Science,DASC17,Customer Sentiment Analysis for Retail And Sales,Ensuring compliance with consumer data protect...,feature toggle dashboard: managed feature togg...,26,TS4276,F4276,...,4,1,7,3,10,6,4,3,5,5
4276,6618e3f4e7506de234ed2c3e,442,Data Science,DASC17,Customer Sentiment Analysis for Retail And Sales,Ensuring compliance with consumer data protect...,security audit: checked dependencies for vulne...,35,TS4277,F4277,...,4,5,7,10,10,9,2,1,6,10
4277,6618e3f4e7506de234ed2c3f,442,Data Science,DASC17,Customer Sentiment Analysis for Retail And Sales,Ensuring compliance with consumer data protect...,code freeze: stabilizing codebase for upcoming...,38,TS4278,F4278,...,10,3,9,10,3,1,10,5,2,7


In [5]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax
from tqdm import tqdm

In [4]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
example = df['additionalComments'][50]
print(example)

The team had difficulty learning from past mistakes, repeating the same errors without reflection.


In [17]:
def polarity_scores_roberta(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores = np.array(scores)
    sentiment_class = ['negative', 'neutral', 'positive'][np.argmax(scores)]
    return scores,sentiment_class

In [18]:
res = {}
for i, row in tqdm(df.iterrows(), total=len(df)):
    try:
        text = row['additionalComments']
        scores, sentiment = polarity_scores_roberta(text)
        res[i] = {'**roberta_result': {'scores': scores, 'sentiment': sentiment}}
    except RuntimeError:
        print(f'Broke for row {i}')

100%|██████████| 4279/4279 [15:11<00:00,  4.69it/s]  


In [19]:
df['sentiment'] = [res[i]['**roberta_result']['sentiment'] for i in range(len(df))]

In [20]:
df.head()

,_id_x,userId_x,projectType_x,projectId_x,projectName_x,task,comment,totalHours,timesheetId,feedbackId,...,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,sentiment
0,66166cc7d31405dcf870afd3,1,Full Stack Web Development,FULL01,Railway Ticket Booking System,"Write unit tests for backend APIs, ensure cod...",good,40,TS01,F01,...,8,7,6,7,2,8,8,8,8,positive
1,66166cc7d31405dcf870afd7,1,Data Engineering,DATA01,DataNexus: Bridging Data Gaps for Holistic Bus...,Define Data Lake Requirements,awesome,47,TS02,F02,...,8,7,9,8,6,6,6,4,1,positive
2,6618e3f4e7506de234ed1b8c,83,Full Stack Web Development,FULL01,Railway Ticket Booking System,"Design the database schema, define tables, and...",patch management: applied security patches.,49,TS03,F03,...,9,2,7,10,4,1,1,7,9,positive
3,6618e3f4e7506de234ed1b8d,83,Full Stack Web Development,FULL01,Railway Ticket Booking System,"Design the database schema, define tables, and...",accessibility audit: fixed wcag compliance iss...,42,TS04,F04,...,7,1,4,8,3,1,3,7,5,neutral
4,6618e3f4e7506de234ed1b8e,83,Full Stack Web Development,FULL01,Railway Ticket Booking System,"Design the database schema, define tables, and...",sprint retrospective: identified areas for imp...,43,TS05,F05,...,6,7,5,8,9,10,8,6,6,neutral


In [21]:
# Save the DataFrame to a CSV file
df.to_csv('sentiment_analysis_output.csv', index=False)